In [1]:
import os
import random

import numpy as np 
from numpy import load
from numpy import savez_compressed
import nibabel as nib
import tensorflow as tf

from nets.unet3d import *
from utils_tumores import *

np.set_printoptions(precision=2, suppress=True)
PARENT_DIR = os.getcwd()
MODEL_PATH = os.path.join(PARENT_DIR, "models", "3D")

BraTS20_Training_001
parent dir:  /media/bersek/56DAFC88DAFC65A1/BRATS


In [2]:
print(output_x)

80


In [3]:
print(tf.__version__)
tf.config.list_physical_devices("GPU") 

2.6.0


2021-10-04 19:33:00.174504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-04 19:33:00.265944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-04 19:33:00.266263: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Crea los directorios si no existen, el unico requisito es que se tenga la DB en una carpeta llamada NFBS_Dataset 
# Dentro del mismo directorio que este codigo
volumes_folders = count_volumes(dataset_folder)
paths, train_files, val_files, test_files = make_dirs(volumes_folders)
paths.keys()


BraTS20_Training_001
parent dir:  /media/bersek/56DAFC88DAFC65A1/BRATS


dict_keys(['SUBVOLUME_FOLDER', 'SUBVOLUME_MASK_FOLDER', 'ROTATION_MATRIX', 'RESULTS_3D', 'DATABASE_DIR', 'SAMPLES', 'TEST_SAMPLES'])

# Data split

In [5]:
print(len(train_files))
print(len(val_files))
print(len(test_files))

295
37
37


In [6]:
print(train_files[0])


BraTS20_Training_344


In [7]:
paths["SUBVOLUME_FOLDER"]

'/media/bersek/56DAFC88DAFC65A1/BRATS/processed/3D/subvolumes'

In [8]:
os.path.join(paths["SUBVOLUME_FOLDER"],"BraTS20_Training_344")

'/media/bersek/56DAFC88DAFC65A1/BRATS/processed/3D/subvolumes/BraTS20_Training_344'

In [9]:
# Dividimos la lista de las carpetas de pacientes en train, val y test
''' train_files, val_files = train_test_split(paths["SAMPLES"], test_size=0.2, random_state=42)
test_files, val_files = train_test_split(val_files, test_size=0.5, random_state=42) '''

train_images=[]
train_images_mask=[]

for sample in train_files:
    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_FOLDER"],sample))):
        train_images.append(os.path.join(paths["SUBVOLUME_FOLDER"],sample,subvol))
        

    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample))):
        train_images_mask.append(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample,subvol))

val_images=[]
val_images_mask=[]
for sample in val_files:
    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_FOLDER"],sample))):
        val_images.append(os.path.join(paths["SUBVOLUME_FOLDER"],sample,subvol))
        

    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample))):
        val_images_mask.append(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample,subvol))


test_images=[]
test_images_mask=[]
# Agregamos una lista mas para crear la ruta de almacenamiendo de las predicciones
test_predict_images=[]

for sample in test_files:
    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_FOLDER"],sample))):
        test_images.append(os.path.join(paths["SUBVOLUME_FOLDER"],sample,subvol))
        test_predict_images.append(os.path.join(paths["RESULTS_3D"],sample,subvol))
        

    for subvol in sorted(os.listdir(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample))):
        test_images_mask.append(os.path.join(paths["SUBVOLUME_MASK_FOLDER"],sample,subvol))


In [10]:
paths["RESULTS_3D"]

'/media/bersek/56DAFC88DAFC65A1/BRATS/processed/3D/subvolumes_predict'

In [11]:
print(len(train_images))
print(len(train_images_mask))

13275
13275


# Dataloader

In [12]:
def load_image(file, label):
    dict_data_file = load(file.numpy().decode('utf-8'))
    dict_data_label = load(label.numpy().decode('utf-8'))
    # extract the first array
    nifti = dict_data_file['arr_0']
    label = dict_data_label['arr_0']
    #nifti = np.asarray(nib.load(file.numpy().decode('utf-8')).get_fdata()).astype(np.int16)
    #label = np.asarray(nib.load(label.numpy().decode('utf-8')).get_fdata()).astype(np.int16)
    return nifti, label


@tf.autograph.experimental.do_not_convert
def load_image_wrapper(file, label):
    image, label = tf.py_function(load_image, [file, label], [tf.float32, tf.float32])
    image.set_shape(tf.TensorShape([output_x, output_y, output_z]))
    label.set_shape(tf.TensorShape([output_x, output_y, output_z]))
    return image, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_images_mask))
train_dataset = train_dataset.shuffle(3, reshuffle_each_iteration=False)
train_dataset = train_dataset.map(load_image_wrapper, num_parallel_calls=32)
train_dataset = train_dataset.batch(8, drop_remainder=False)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_images_mask))
val_dataset = val_dataset.shuffle(3, reshuffle_each_iteration=False)
val_dataset = val_dataset.map(load_image_wrapper, num_parallel_calls=32)
val_dataset = val_dataset.batch(8, drop_remainder=False)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_images_mask))
test_dataset = test_dataset.map(load_image_wrapper, num_parallel_calls=32)
test_dataset = test_dataset.batch(8, drop_remainder=False)

2021-10-04 19:33:00.852625: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-04 19:33:00.852896: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-04 19:33:00.853251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-04 19:33:00.853531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [13]:
opt = keras.optimizers.Adam(learning_rate=0.001)

In [14]:
model_unet=unet_3D(output_x, output_y, output_z)
model_unet.compile(optimizer=opt, loss = 'binary_crossentropy', 
                    metrics = ['accuracy', dice_coeff]) 

(None, 80, 80, 32, 1) 

(None, 80, 80, 32, 32)
(None, 80, 80, 32, 32)
(None, 40, 40, 16, 32)
(None, 40, 40, 16, 64)
(None, 40, 40, 16, 64)
(None, 20, 20, 8, 64)
(None, 20, 20, 8, 128)
(None, 20, 20, 8, 128)
(None, 10, 10, 4, 128)
(None, 10, 10, 4, 256)
(None, 10, 10, 4, 256)  here go1 

(None, 5, 5, 2, 256)  here go
 
(None, 5, 5, 2, 512)
(None, 5, 5, 2, 512) 

(None, 10, 10, 4, 256)
(None, 10, 10, 4, 512)
(None, 10, 10, 4, 256)
(None, 10, 10, 4, 256)
(None, 20, 20, 8, 128)
(None, 20, 20, 8, 256)
(None, 20, 20, 8, 128)
(None, 20, 20, 8, 128)
(None, 40, 40, 16, 64)
(None, 40, 40, 16, 128)
(None, 40, 40, 16, 64)
(None, 40, 40, 16, 64)
(None, 80, 80, 32, 32)
(None, 80, 80, 32, 64)
(None, 80, 80, 32, 32)
(None, 80, 80, 32, 32) 

(None, 80, 80, 32, 1)


In [15]:
callbacks = ModelCheckpoint(MODEL_PATH, 
                            monitor='val_dice_coeff',
                            verbose=1, 
                            mode='max',
                            save_best_only=True)

In [16]:
history = model_unet.fit(train_dataset,
                         validation_data=val_dataset,
                         epochs=3,
                         callbacks=[callbacks, tf.keras.callbacks.EarlyStopping(verbose=2, patience=2,min_delta=0.001, monitor='val_dice_coeff')]) #Guardar la mejor epoca para validación

2021-10-04 19:33:02.590773: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3


2021-10-04 19:33:05.596007: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202
2021-10-04 19:33:06.930018: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1660/1660 [==============================] - 1296s 775ms/step - loss: 0.1172 - accuracy: 0.9820 - dice_coeff: 0.6474 - val_loss: 0.0762 - val_accuracy: 0.9887 - val_dice_coeff: 0.7859

Epoch 00001: val_dice_coeff improved from -inf to 0.78589, saving model to /media/bersek/56DAFC88DAFC65A1/BRATS/code/models/3D


2021-10-04 19:54:40.934274: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /media/bersek/56DAFC88DAFC65A1/BRATS/code/models/3D/assets
Epoch 2/3
1660/1660 [==============================] - 1288s 776ms/step - loss: 0.0633 - accuracy: 0.9862 - dice_coeff: 0.6353 - val_loss: 0.1256 - val_accuracy: 0.9359 - val_dice_coeff: 0.4873

Epoch 00002: val_dice_coeff did not improve from 0.78589
Epoch 3/3
1660/1660 [==============================] - 1207s 727ms/step - loss: 0.0545 - accuracy: 0.9839 - dice_coeff: 0.5988 - val_loss: 0.2112 - val_accuracy: 0.9546 - val_dice_coeff: 0.5316

Epoch 00003: val_dice_coeff did not improve from 0.78589


In [18]:
results = model_unet.evaluate(test_dataset)


209/209 [==============================] - 45s 213ms/step - loss: 0.2024 - accuracy: 0.9595 - dice_coeff: 0.5590


# Predicciones

In [24]:
model_unet_predict=unet_3D(output_x, output_y, output_z)

(None, 80, 80, 32, 1) 

(None, 80, 80, 32, 32)
(None, 80, 80, 32, 32)
(None, 40, 40, 16, 32)
(None, 40, 40, 16, 64)
(None, 40, 40, 16, 64)
(None, 20, 20, 8, 64)
(None, 20, 20, 8, 128)
(None, 20, 20, 8, 128)
(None, 10, 10, 4, 128)
(None, 10, 10, 4, 256)
(None, 10, 10, 4, 256)  here go1 

(None, 5, 5, 2, 256)  here go
 
(None, 5, 5, 2, 512)
(None, 5, 5, 2, 512) 

(None, 10, 10, 4, 256)
(None, 10, 10, 4, 512)
(None, 10, 10, 4, 256)
(None, 10, 10, 4, 256)
(None, 20, 20, 8, 128)
(None, 20, 20, 8, 256)
(None, 20, 20, 8, 128)
(None, 20, 20, 8, 128)
(None, 40, 40, 16, 64)
(None, 40, 40, 16, 128)
(None, 40, 40, 16, 64)
(None, 40, 40, 16, 64)
(None, 80, 80, 32, 32)
(None, 80, 80, 32, 64)
(None, 80, 80, 32, 32)
(None, 80, 80, 32, 32) 

(None, 80, 80, 32, 1)


In [25]:
model_unet_predict.load_weights(MODEL_PATH)

2021-10-04 20:51:27.354348: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open /media/bersek/56DAFC88DAFC65A1/BRATS/code/models/3D: Failed precondition: /media/bersek/56DAFC88DAFC65A1/BRATS/code/models/3D; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [28]:
for sub_volume in range(len(test_images)):
    #Cargamos cada imagen como un array
    dict_data = load(test_images[sub_volume])
    # extract the first array
    nifti = dict_data['arr_0']
    #Convertimos a tensor
    image = tf.convert_to_tensor(nifti, dtype=tf.float32)
    #Agregamos la dimension del batch y no recuerdo cual es la otra
    image = tf.expand_dims(image, axis=3)
    image = tf.expand_dims(image, axis=0)
    #Se realiza la prediccion
    prediction = model_unet.predict(image)
    #Elimina la dimension del batch
    prediction = prediction.squeeze()
    #Aplicamos un threshold para el valor de cada pixel
    prediction[prediction>=0.5]=1.0
    prediction[prediction<0.5]=0.0
    #Guardamos la imagen
    savez_compressed(test_predict_images[sub_volume], prediction)
    

In [21]:
test_files

['BraTS20_Training_345',
 'BraTS20_Training_223',
 'BraTS20_Training_139',
 'BraTS20_Training_302',
 'BraTS20_Training_368',
 'BraTS20_Training_272',
 'BraTS20_Training_064',
 'BraTS20_Training_324',
 'BraTS20_Training_244',
 'BraTS20_Training_073',
 'BraTS20_Training_308',
 'BraTS20_Training_274',
 'BraTS20_Training_184',
 'BraTS20_Training_294',
 'BraTS20_Training_217',
 'BraTS20_Training_006',
 'BraTS20_Training_249',
 'BraTS20_Training_304',
 'BraTS20_Training_221',
 'BraTS20_Training_042',
 'BraTS20_Training_045',
 'BraTS20_Training_065',
 'BraTS20_Training_212',
 'BraTS20_Training_024',
 'BraTS20_Training_366',
 'BraTS20_Training_211',
 'BraTS20_Training_199',
 'BraTS20_Training_036',
 'BraTS20_Training_282',
 'BraTS20_Training_169',
 'BraTS20_Training_017',
 'BraTS20_Training_230',
 'BraTS20_Training_010',
 'BraTS20_Training_118',
 'BraTS20_Training_185',
 'BraTS20_Training_142',
 'BraTS20_Training_132']

# Reconstruccion de los volumenes de test

In [29]:
#Esta funcion esta muy bien documentada en el modulo utils.py
pathe=reconstruction(test_files=test_files, path_resultados=paths["RESULTS_3D"],path_rotation_matrix=paths["ROTATION_MATRIX"])

X: 3  Y: 3  Z: 5
